In [1]:
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import pyqpanda as pq
import pyvqnet as pv
import random

%matplotlib inline

In [2]:
train_data = Dataset(
    "data/cifar-10-batches-py/data_batch_1",
    "data/cifar-10-batches-py/data_batch_2",
    "data/cifar-10-batches-py/data_batch_3",
    "data/cifar-10-batches-py/data_batch_4",
    "data/cifar-10-batches-py/data_batch_5",
)
train_data.enhance()
test_data = Dataset("data/cifar-10-batches-py/test_batch")

In [3]:
class Model(pv.nn.module.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = pv.nn.Conv2D(3, 64, (3, 3), (1, 1))
        self.pool1 = pv.nn.MaxPool2D((2, 2), (2, 2))
        self.conv2 = pv.nn.Conv2D(64, 128, (4, 4), (1, 1))
        self.pool2 = pv.nn.MaxPool2D((2, 2), (2, 2))
        self.conv3 = pv.nn.Conv2D(128, 32, (5, 5), (1, 1))
        self.pool3 = pv.nn.MaxPool2D((2, 2), (2, 2))
        self.li = pv.nn.Linear(32 * 1 * 1, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = pv.tensor.flatten(x, 1)
        x = self.li(x)
        return x

In [4]:
epoch = 10
batch = 10
holdout = 10000

model = Model()

X, Y = train_data.getdatas()
los = pv.nn.loss.CrossEntropyLoss()
opt = pv.optim.SGD(model.parameters(), 0.01)

for e in range(1, epoch + 1):
    model.train()

    correct_tot = 0
    for I, (x, y) in enumerate(
        pv.data.data_generator(
            X[:-holdout], Y[:-holdout], batch_size=batch, shuffle=True
        )
    ):
        i = I + 1
        opt.zero_grad()
        y_pred = model(x)
        loss = los(y, y_pred)
        correct = sum(y_pred.argmax(1, False).to_numpy() == y)
        acc = correct / y.shape[0]
        correct_tot += correct
        loss.backward()
        opt._step()
        if i % 100 == 0:
            print(
                f"epoch {e} \t batch {batch}*{i} \t loss {loss.item():.2f} \t accuracy {acc:.2f}"
            )

    model.eval()

    acc = correct_tot / (Y.shape[0] - holdout)
    print(f"********** train: epoch {e} \t accuracy {100*acc:.2f}% **********")

    correct_tot = 0
    for x, y in pv.data.data_generator(
        X[-holdout:], Y[-holdout:], batch_size=batch, shuffle=False
    ):
        y_pred = model(x)
        correct_tot += sum(y_pred.argmax(1, False).to_numpy() == y)

    acc = correct_tot / holdout
    print(f"********** test: epoch {e} \t accuracy {100*acc:.2f}% **********")

epoch 1 	 batch 10*100 	 loss 2.21 	 accuracy 0.40
epoch 1 	 batch 10*200 	 loss 2.24 	 accuracy 0.10
epoch 1 	 batch 10*300 	 loss 1.84 	 accuracy 0.30
epoch 1 	 batch 10*400 	 loss 1.88 	 accuracy 0.30
epoch 1 	 batch 10*500 	 loss 2.09 	 accuracy 0.00
epoch 1 	 batch 10*600 	 loss 1.92 	 accuracy 0.20
epoch 1 	 batch 10*700 	 loss 1.92 	 accuracy 0.30
epoch 1 	 batch 10*800 	 loss 1.41 	 accuracy 0.60
epoch 1 	 batch 10*900 	 loss 1.64 	 accuracy 0.40
epoch 1 	 batch 10*1000 	 loss 1.75 	 accuracy 0.20
epoch 1 	 batch 10*1100 	 loss 1.79 	 accuracy 0.50
epoch 1 	 batch 10*1200 	 loss 1.98 	 accuracy 0.20
epoch 1 	 batch 10*1300 	 loss 2.19 	 accuracy 0.20
epoch 1 	 batch 10*1400 	 loss 1.53 	 accuracy 0.50
epoch 1 	 batch 10*1500 	 loss 1.51 	 accuracy 0.30
epoch 1 	 batch 10*1600 	 loss 1.55 	 accuracy 0.40
epoch 1 	 batch 10*1700 	 loss 1.83 	 accuracy 0.40
epoch 1 	 batch 10*1800 	 loss 1.47 	 accuracy 0.30
epoch 1 	 batch 10*1900 	 loss 1.44 	 accuracy 0.30
epoch 1 	 batch 10*20

In [5]:
pv.utils.storage.save_parameters(model.state_dict(), "train5.model")

In [6]:
m = Model()
m.load_state_dict(pv.utils.storage.load_parameters("train5.model"))

In [7]:
m.eval()
X, Y = test_data.getdatas()
correct_tot = 0
for x, y in pv.data.data_generator(X, Y, batch_size=batch, shuffle=False):
    y_pred = model(x)
    correct_tot += sum(y_pred.argmax(1, False).to_numpy() == y)
acc = correct_tot / Y.shape[0]
print(f"********** eval: accuracy {100*acc:.2f}% **********")

********** eval: accuracy 69.74% **********
